In [5]:
from llama_cpp import Llama

# Load the model on GPU
llm = Llama(
  model_path="./qwen2.5-7b-instruct-q4_k_m.gguf",
  n_gpu_layers=9999,
  seed=23,
  n_ctx=4000,
  verbose=False
)

llama_init_from_model: n_ctx_per_seq (4000) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [9]:
from pydantic import BaseModel
from typing import List, Dict, Any, Optional

class AgentTool:
  def __init__(self, name, description, input_schema, output_schema):
    self.name = name
    self.description = description
    self.input_schema = input_schema
    self.output_schema = output_schema
  def __call__(self):
    return None
  def get_tool_input(self, text_input):
    return None

In [12]:
class ChatbotInput(BaseModel):
  text: str

class ChatbotOutput(BaseModel):
  response: str

class Chatbot(AgentTool):
  def __init__(self):
    super().__init__(
      name="Chatbot",
      description="An intelligent AI chatbot that can generate human-like responses to text input and can be used to provide information, answer questions, and engage in conversation.",
      input_schema=ChatbotInput,
      output_schema=ChatbotOutput
    )
  def __call__(self, text):
    chatbot_input = self.get_tool_input(text)
    text = chatbot_input.text
    response = llm.create_chat_completion(
      messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {
          "role": "user",
          "content": text
        }
      ],
      max_tokens=1000,
    )
    response = response['choices'][0]['message']['content']
    return ChatbotOutput(response=response)

  def get_tool_input(self, text_input):
    return ChatbotInput(text=text_input)

In [13]:
# Test the chatbot
chatbot = Chatbot()
response = chatbot("Hello, how are you?")
print(response)

response="Hello! I'm just a computer program, so I don't have feelings or consciousness. But I'm here and ready to help you with any questions or conversations you'd like to have. How can I assist you today?"


In [14]:
class CalculatorInput(BaseModel):
  operation: str
  a: float
  b: float

class CalculatorOutput(BaseModel):
  result: Optional[float]

class Calculator(AgentTool):
  def __init__(self):
    super().__init__(
      name="Calculator",
      description="A simple calculator tool for arithmetic operations.",
      input_schema=CalculatorInput,
      output_schema=CalculatorOutput
    )
    
  def __call__(self, input_text):
    calculator_input = self.get_tool_input(input_text)
    operation = calculator_input.operation
    a = calculator_input.a
    b = calculator_input.b
    result = None
    if operation == "add":
      result = self.add(a, b)
    elif operation == "subtract":
      result = self.subtract(a, b)
    elif operation == "multiply":
      result = self.multiply(a, b)
    elif operation == "divide":
      result = self.divide(a, b)
    else:
      result = None

    return CalculatorOutput(result=result)

  def get_tool_input(self, input_text):
    response = llm.create_chat_completion(
      messages = [
        {"role": "system", "content": "You are a helpful assistant who excels at extracting the operation and numbers for an arithmetic operation from the user input. Return your response in the following XML format: <response><operation>{'add', 'subtract', 'multiply' or 'divide'}</operation><a>{operand a}</a><b>{operand b}</b></response>"},
        {
          "role": "user",
          "content": f"Please extract the operation and numbers from the following text: <text>{input_text}</text>. <response>"
        }
      ]
    )
    response_text = response['choices'][0]['message']['content']
    operation = response_text.split("<operation>")[1].split("</operation>")[0]
    a = float(response_text.split("<a>")[1].split("</a>")[0])
    b = float(response_text.split("<b>")[1].split("</b>")[0])
    return self.input_schema(operation=operation, a=a, b=b)
    
  def add(self, a, b):
    return a + b

  def subtract(self, a, b):
    return a - b

  def multiply(self, a, b):
    return a * b

  def divide(self, a, b):
    return a / b

In [15]:
# Test calculator tool
calculator = Calculator()
calculator("What is 5 plus 3?")

CalculatorOutput(result=8.0)

In [19]:
import re

class Agent:
  def __init__(self, tools):
    self.tools = tools

  def __call__(self, input):
    tool = self.get_tool(input, self.tools)
    if tool is None:
      return "I am unable to help you."
    else:
      response = tool(input)
      return response

  def get_tool(self, input: str, tools: list):
    # create json string with tools with their names and descriptions
    tools_dict = [
      {"name": tool.name, "description": tool.description}
      for tool in tools
    ]

    response = llm.create_chat_completion(
      messages = [
        {"role": "system", "content": """You are an intelligent AI agent that can make decisions and use tools to solve problems. 
        Given the following problem, please select the most appropriate tool to solve it. 
        Think step-by-step before making a decision.
        Provide your response in the following format: <think>step-by-step reasoning</think> <tool>tool name</tool>.
        The tool name provided should be one of the tools given in exact."""},
        {
          "role": "user",
          "content": f"Given the following tools: <tools>{tools}</tools>, and the following problem: <problem>{input}</problem>, which tool would you use to solve the problem?"
        }
      ]
    )

    print("get_tool response:", response)

    # extract tool name from response
    tool_name = re.search(r'<tool>(.*)</tool>', response['choices'][0]['message']['content']).group(1)

    # get tool object from tool name
    for tool in tools:
      if tool.name == tool_name:
        return tool
      
    return None

In [20]:
tools = [chatbot, calculator]
print(tools[0].name)
print(tools[0].description)

Chatbot
An intelligent AI chatbot that can generate human-like responses to text input and can be used to provide information, answer questions, and engage in conversation.


In [21]:
# Test agent
agent = Agent(tools=tools)

print("What is 3x3?: ", agent("What is 3x3?"))

get_tool response: {'id': 'chatcmpl-47305cb9-1a7e-4c40-853a-10270baa4b3e', 'object': 'chat.completion', 'created': 1742010478, 'model': './qwen2.5-7b-instruct-q4_k_m.gguf', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '<think>Step-by-step reasoning:\n1. The problem is a simple multiplication question: 3x3.\n2. We have two tools available: a Chatbot and a Calculator.\n3. The Chatbot is not designed to perform mathematical calculations.\n4. The Calculator is designed to perform mathematical operations, which includes multiplication.\n5. Therefore, the most appropriate tool to solve the problem is the Calculator.</think>\n<tool>Calculator</tool>'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 187, 'completion_tokens': 91, 'total_tokens': 278}}
What is 3x3?:  result=9.0


In [22]:
print("What matter is water: ", agent("What matter is water?"))

get_tool response: {'id': 'chatcmpl-15d5813b-d92e-469a-9329-2cdf7d74efe3', 'object': 'chat.completion', 'created': 1742010480, 'model': './qwen2.5-7b-instruct-q4_k_m.gguf', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '<think>Step-by-step reasoning:\n1. The problem "What matter is water?" is asking about the composition or nature of water.\n2. The tools available are a Chatbot and a Calculator.\n3. A Chatbot can provide conversational and informational responses, which might include scientific facts about water.\n4. A Calculator is used for mathematical operations and does not provide scientific information.\n5. Since the question is about the nature of water, a scientific or informational response is needed.</think>\n<tool>Chatbot</tool>'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 185, 'completion_tokens': 108, 'total_tokens': 293}}
What matter is water:  response='Water is a chemical substance with the chemical formula H₂O. It co